# Unsupervised Learning for Clustering NBA Players
---
## Web Scraping
Data for this project was scraped from the popular NBA stats tracking website https://www.basketball-reference.com/.

Remy Shea, May 2019, https://github.com/RemShea/nba-player-clustering

---
# Import Libraries

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import requests
from bs4 import BeautifulSoup
import time

%matplotlib inline

In [5]:
url='https://www.basketball-reference.com/'
res=requests.get(url)
soup=BeautifulSoup(res.content,'lxml')

---
# Gathering Team URLs

In [7]:
west = soup.find('table',{'id':'confs_standings_W'})
east = soup.find('table',{'id':'confs_standings_E'})
teams=[]
team_urls=[]
for row in west.find_all('tr')[1:]:
    team_link = row.find('a')['href']
    team_url = 'https://www.basketball-reference.com' + team_link
    team_urls.append(team_url)
for row in east.find_all('tr')[1:]:
    team_link = row.find('a')['href']
    team_url = 'https://www.basketball-reference.com' + team_link
    team_urls.append(team_url)

---
# Gathering Player URLs

In [8]:
%%time
players = dict()
for url in team_urls:
    print(f'Scraping data from {url}...')
    res=requests.get(url)
    soup=BeautifulSoup(res.content,'lxml')
    table=soup.find('table')
    links=table.find_all('a')
    for i in links:
        link=i['href']
        name=i.text
        if 'html' in link:
            players[name]=dict()
            players[name]['url']=('https://www.basketball-reference.com' + link)
    time.sleep(1)
print('Finished collecting roster data!\n'+'-'*60)

Scraping data from https://www.basketball-reference.com/teams/GSW/2019.html...
Scraping data from https://www.basketball-reference.com/teams/DEN/2019.html...
Scraping data from https://www.basketball-reference.com/teams/POR/2019.html...
Scraping data from https://www.basketball-reference.com/teams/HOU/2019.html...
Scraping data from https://www.basketball-reference.com/teams/UTA/2019.html...
Scraping data from https://www.basketball-reference.com/teams/OKC/2019.html...
Scraping data from https://www.basketball-reference.com/teams/SAS/2019.html...
Scraping data from https://www.basketball-reference.com/teams/LAC/2019.html...
Scraping data from https://www.basketball-reference.com/teams/SAC/2019.html...
Scraping data from https://www.basketball-reference.com/teams/LAL/2019.html...
Scraping data from https://www.basketball-reference.com/teams/MIN/2019.html...
Scraping data from https://www.basketball-reference.com/teams/MEM/2019.html...
Scraping data from https://www.basketball-reference.

---
# Gathering Player Data

In [10]:
MIN_GAMES=10
MIN_MINUTES=500

failed_to_qualify=[]
for i,player in enumerate(players):
    start = time.time()
    url = players[player]['url']
    players[player]['name']=player
    print(f'Collecting stats for {player} from {url} ...')
    res=requests.get(url)
    soup=BeautifulSoup(res.content,'lxml')
    
    try:
        per_game_stats = soup.find('table',{'id':'per_game'}).find('tr',{'id':'per_game.2019'}).find_all('td',{'class':'right'})
    except AttributeError:
        print(f'{player} didn\'t play in an NBA game in 2019! YEET!')
        failed_to_qualify.append(player)
        continue
        
    for stat in per_game_stats:
        if stat.text != '':
            stat_name=stat['data-stat']
            players[player][stat_name]=float(stat.text)
    
    advanced_stats = BeautifulSoup(str(soup.find('div',{'id':'all_advanced'})).replace('<!--','')).find('table',{'id':'advanced'}).find('tr',{'id':'advanced.2019'}).find_all('td',{'class':'right'})
    for stat in advanced_stats:
        if stat.text != '':
            stat_name=stat['data-stat']
            players[player][stat_name]=float(stat.text)
            
    shooting_stats = BeautifulSoup(str(soup.find('div',{'id':'all_shooting'})).replace('<!--','')).find('table',{'id':'shooting'}).find('tr',{'id':'shooting.2019'}).find_all('td',{'class':'right'})
    for stat in shooting_stats:
        if stat.text!='':
            stat_name=stat['data-stat']
            players[player][stat_name]=float(stat.text)
            
    pbp_stats = BeautifulSoup(str(soup.find('div',{'id':'all_pbp'})).replace('<!--','')).find('table',{'id':'pbp'}).find('tr',{'id':'pbp.2019'}).find_all('td',{'class':'right'})
    for stat in pbp_stats:
        if stat.text!='':
            stat_name=stat['data-stat']
            if stat.text.endswith('%'):
                text = stat.text.replace('%','')
            else:
                text = stat.text
            players[player][stat_name]=float(text)
        
    if players[player]['g'] < MIN_GAMES or players[player]['mp'] < MIN_MINUTES:
        print(f'{player} failed to meet for minutes or games played minimum! YEET!')
        failed_to_qualify.append(player)
        
    time.sleep(3)
    end=time.time()
    print(f'ETA: {(len(players)-(i+1))*(end-start)} seconds...')
print('Finished collecting player data!\n'+'-'*60)

ETA: 1828.7703475952148 seconds...
ETA: 1825.1189517974854 seconds...
Damian Jones failed to meet for minutes or games played minimum! YEET!
ETA: 1755.4769048690796 seconds...
ETA: 1750.8032658100128 seconds...
ETA: 1874.4748106002808 seconds...
ETA: 1843.2811691761017 seconds...
ETA: 1896.290364265442 seconds...
ETA: 1790.8504135608673 seconds...
Jacob Evans failed to meet for minutes or games played minimum! YEET!
ETA: 1801.3763484954834 seconds...
ETA: 1815.3777060508728 seconds...
ETA: 1854.3441338539124 seconds...
ETA: 1930.9823596477509 seconds...
ETA: 1857.4371690750122 seconds...
Damion Lee failed to meet for minutes or games played minimum! YEET!
ETA: 1665.3961844444275 seconds...
Marcus Derrickson failed to meet for minutes or games played minimum! YEET!
ETA: 1651.6693406105042 seconds...
ETA: 1707.722110748291 seconds...
Andrew Bogut failed to meet for minutes or games played minimum! YEET!
ETA: 1781.6432189941406 seconds...
ETA: 1807.8023822307587 seconds...
ETA: 1714.04724

ETA: 1646.0064613819122 seconds...
ETA: 1753.7365064620972 seconds...
ETA: 1658.8625783920288 seconds...
ETA: 1692.1095480918884 seconds...
ETA: 1704.437667131424 seconds...
Trevon Duval failed to meet for minutes or games played minimum! YEET!
ETA: 1525.2911758422852 seconds...
Isaiah Hartenstein failed to meet for minutes or games played minimum! YEET!
ETA: 1561.4906239509583 seconds...
ETA: 1557.7114763259888 seconds...
ETA: 1552.017725944519 seconds...
Vince Edwards failed to meet for minutes or games played minimum! YEET!
ETA: 1628.609642982483 seconds...
ETA: 1580.6839299201965 seconds...
ETA: 1600.2083039283752 seconds...
ETA: 1588.0085129737854 seconds...
Chris Chiozza failed to meet for minutes or games played minimum! YEET!
ETA: 1511.4885177612305 seconds...
Michael Frazier didn't play in an NBA game in 2019! YEET!
ETA: 1534.123878479004 seconds...
ETA: 1600.4039261341095 seconds...
ETA: 1602.4345331192017 seconds...
ETA: 1508.1076278686523 seconds...
Tony Bradley failed to m

ETA: 1408.3901381492615 seconds...
ETA: 1396.4788134098053 seconds...
ETA: 1439.1676120758057 seconds...
ETA: 1520.1684784889221 seconds...
ETA: 1444.4398641586304 seconds...
Lonnie Walker failed to meet for minutes or games played minimum! YEET!
ETA: 1393.2516553401947 seconds...
ETA: 1462.7845067977905 seconds...
ETA: 1496.216251373291 seconds...
ETA: 1378.0161089897156 seconds...
Quincy Pondexter failed to meet for minutes or games played minimum! YEET!
ETA: 1433.5412633419037 seconds...
Chimezie Metu failed to meet for minutes or games played minimum! YEET!
ETA: 1334.0520629882812 seconds...
Drew Eubanks failed to meet for minutes or games played minimum! YEET!
ETA: 1330.4378929138184 seconds...
Ben Moore didn't play in an NBA game in 2019! YEET!
Donatas Motiejunas failed to meet for minutes or games played minimum! YEET!
ETA: 1423.5768055915833 seconds...
ETA: 1422.8172874450684 seconds...
ETA: 1429.286675453186 seconds...
ETA: 1463.0045127868652 seconds...
ETA: 1354.5662786960602

ETA: 1278.5688779354095 seconds...
ETA: 1261.6697630882263 seconds...
Moritz Wagner failed to meet for minutes or games played minimum! YEET!
ETA: 1210.6430730819702 seconds...
Isaac Bonga failed to meet for minutes or games played minimum! YEET!
ETA: 1190.860071182251 seconds...
ETA: 1220.082329750061 seconds...
ETA: 1276.8142309188843 seconds...
ETA: 1353.3789143562317 seconds...
ETA: 1274.1774673461914 seconds...
ETA: 1266.5019822120667 seconds...
ETA: 1185.4629201889038 seconds...
Johnathan Williams failed to meet for minutes or games played minimum! YEET!
ETA: 1187.8992562294006 seconds...
ETA: 1287.9395914077759 seconds...
Jemerrio Jones failed to meet for minutes or games played minimum! YEET!
ETA: 1149.1949496269226 seconds...
ETA: 1202.078161239624 seconds...
ETA: 1177.3346683979034 seconds...
ETA: 1214.2982749938965 seconds...
ETA: 1167.340627670288 seconds...
ETA: 1160.9401316642761 seconds...
ETA: 1164.075928926468 seconds...
ETA: 1261.86141872406 seconds...
ETA: 1196.31029

ETA: 1096.0339698791504 seconds...
ETA: 1031.4082517623901 seconds...
ETA: 1059.8189687728882 seconds...
ETA: 1070.5124304294586 seconds...
ETA: 1009.4780044555664 seconds...
ETA: 1037.515061378479 seconds...
ETA: 1080.5784029960632 seconds...
ETA: 1028.9699685573578 seconds...
ETA: 1057.5805130004883 seconds...
Trevon Bluiett didn't play in an NBA game in 2019! YEET!
ETA: 981.9265866279602 seconds...
ETA: 1024.6097857952118 seconds...
Christian Wood failed to meet for minutes or games played minimum! YEET!
ETA: 1000.2302360534668 seconds...
Dairis Bertans failed to meet for minutes or games played minimum! YEET!
ETA: 970.2626667022705 seconds...
Kristaps Porzingis didn't play in an NBA game in 2019! YEET!
ETA: 1055.344293832779 seconds...
ETA: 1020.3833913803101 seconds...
ETA: 979.1080355644226 seconds...
Courtney Lee failed to meet for minutes or games played minimum! YEET!
ETA: 1038.7917838096619 seconds...
ETA: 971.1516466140747 seconds...
ETA: 994.6466255187988 seconds...
ETA: 96

ETA: 870.4873900413513 seconds...
Donte DiVincenzo failed to meet for minutes or games played minimum! YEET!
ETA: 823.6817157268524 seconds...
ETA: 917.1958541870117 seconds...
ETA: 865.3120589256287 seconds...
ETA: 845.2438144683838 seconds...
Bonzie Colson failed to meet for minutes or games played minimum! YEET!
ETA: 816.2460570335388 seconds...
Pau Gasol failed to meet for minutes or games played minimum! YEET!
ETA: 926.2720737457275 seconds...
ETA: 862.3294529914856 seconds...
ETA: 858.7149262428284 seconds...
ETA: 848.6872878074646 seconds...
ETA: 870.838002204895 seconds...
ETA: 804.9776666164398 seconds...
ETA: 801.1838111877441 seconds...
ETA: 798.6676454544067 seconds...
ETA: 860.1197128295898 seconds...
ETA: 833.75017786026 seconds...
ETA: 788.6700868606567 seconds...
Chris Boucher failed to meet for minutes or games played minimum! YEET!
ETA: 769.561384677887 seconds...
Jordan Loyd failed to meet for minutes or games played minimum! YEET!
ETA: 763.0724430084229 seconds...
P

Robert Williams failed to meet for minutes or games played minimum! YEET!
ETA: 668.5143070220947 seconds...
ETA: 694.2565298080444 seconds...
ETA: 674.0590076446533 seconds...
PJ Dozier failed to meet for minutes or games played minimum! YEET!
ETA: 639.393789768219 seconds...
R.J. Hunter failed to meet for minutes or games played minimum! YEET!
ETA: 652.6146764755249 seconds...
Jonathan Gibson didn't play in an NBA game in 2019! YEET!
ETA: 669.8231649398804 seconds...
ETA: 636.0510532855988 seconds...
ETA: 682.3677368164062 seconds...
ETA: 663.5069696903229 seconds...
ETA: 639.7103576660156 seconds...
ETA: 648.8728702068329 seconds...
ETA: 636.7994842529297 seconds...
ETA: 657.950136423111 seconds...
Edmond Sumner failed to meet for minutes or games played minimum! YEET!
ETA: 618.7916669845581 seconds...
ETA: 595.1677393913269 seconds...
ETA: 671.1474275588989 seconds...
ETA: 627.2411108016968 seconds...
Kyle O'Quinn failed to meet for minutes or games played minimum! YEET!
ETA: 615.94

Troy Caupain failed to meet for minutes or games played minimum! YEET!
ETA: 465.5567307472229 seconds...
Amile Jefferson failed to meet for minutes or games played minimum! YEET!
ETA: 462.9602138996124 seconds...
Michael Carter-Williams failed to meet for minutes or games played minimum! YEET!
ETA: 486.8083162307739 seconds...
ETA: 482.48925256729126 seconds...
Jon Leuer failed to meet for minutes or games played minimum! YEET!
ETA: 488.4938359260559 seconds...
ETA: 492.9239909648895 seconds...
ETA: 458.2199401855469 seconds...
ETA: 465.9226264953613 seconds...
ETA: 448.15567445755005 seconds...
ETA: 450.8582651615143 seconds...
ETA: 463.00940322875977 seconds...
ETA: 427.71208477020264 seconds...
ETA: 448.637170791626 seconds...
ETA: 459.9474685192108 seconds...
Sviatoslav Mykhailiuk failed to meet for minutes or games played minimum! YEET!
ETA: 419.77309226989746 seconds...
ETA: 450.8323311805725 seconds...
Khyri Thomas failed to meet for minutes or games played minimum! YEET!
ETA: 4

ETA: 299.0358829498291 seconds...
ETA: 290.41001200675964 seconds...
ETA: 295.1656723022461 seconds...
Ian Mahinmi failed to meet for minutes or games played minimum! YEET!
ETA: 292.3168423175812 seconds...
ETA: 276.98033380508423 seconds...
ETA: 280.5291659832001 seconds...
Devin Robinson failed to meet for minutes or games played minimum! YEET!
ETA: 265.16110706329346 seconds...
ETA: 268.50675344467163 seconds...
ETA: 286.9489245414734 seconds...
ETA: 277.1296863555908 seconds...
Dwight Howard failed to meet for minutes or games played minimum! YEET!
ETA: 282.08923530578613 seconds...
ETA: 255.49810528755188 seconds...
ETA: 244.57782745361328 seconds...
Jordan McRae failed to meet for minutes or games played minimum! YEET!
ETA: 247.6387288570404 seconds...
ETA: 238.4400224685669 seconds...
Tarik Phillip didn't play in an NBA game in 2019! YEET!
Justin Anderson failed to meet for minutes or games played minimum! YEET!
ETA: 242.48902130126953 seconds...
ETA: 238.36580514907837 seconds.

ETA: 114.10638785362244 seconds...
ETA: 111.6877555847168 seconds...
ETA: 105.97687029838562 seconds...
ETA: 100.40110969543457 seconds...
ETA: 99.18914937973022 seconds...
Marquese Chriss failed to meet for minutes or games played minimum! YEET!
ETA: 92.87187051773071 seconds...
J.R. Smith failed to meet for minutes or games played minimum! YEET!
ETA: 97.5690245628357 seconds...
ETA: 85.81154823303223 seconds...
ETA: 81.46592998504639 seconds...
ETA: 76.38316869735718 seconds...
Channing Frye failed to meet for minutes or games played minimum! YEET!
ETA: 79.19433760643005 seconds...
ETA: 72.84680366516113 seconds...
Jaron Blossomgame failed to meet for minutes or games played minimum! YEET!
ETA: 66.89237093925476 seconds...
Deng Adel failed to meet for minutes or games played minimum! YEET!
ETA: 62.67787313461304 seconds...
ETA: 61.45279121398926 seconds...
ETA: 57.29159927368164 seconds...
ETA: 55.302947759628296 seconds...
ETA: 52.60061311721802 seconds...
ETA: 45.56796455383301 sec

---
# Data Dictionary Mock-Up

In [335]:
data_dict=dict()
tables = ['all_per_game','all_advanced','all_shooting','all_pbp']
for table in tables:
    my_soup = BeautifulSoup(str(soup.find('div',{'id':table})).replace('<!--',''))
    for th in my_soup.find_all('th',{'class':'poptip'}):
        var_name = th['data-stat']
        stat_name = th['aria-label']
        try:
            stat_desc = th['data-tip']
            data_dict[var_name]=stat_name + ' : ' + stat_desc
        except KeyError:
            data_dict[var_name]=stat_name

In [340]:
for key,value in data_dict.items():
    print(key + '\n' + value)
    print('-'*60+'\n')  

season
If listed as single number, the year the season ended.★ - Indicates All-Star for league.Only on regular season tables. : If listed as single number, the year the season ended.<br>★ - Indicates All-Star for league.<br>Only on regular season tables.
------------------------------------------------------------

age
Age of Player at the start of February 1st of that season. : Age of Player at the start of February 1st of that season.
------------------------------------------------------------

team_id
Team : Team
------------------------------------------------------------

lg_id
League : League
------------------------------------------------------------

pos
Position : Position
------------------------------------------------------------

g
Games : Games
------------------------------------------------------------

gs
Games Started : Games Started
------------------------------------------------------------

mp_per_g
Minutes Played Per Game : Minutes Played Per Game
-------------

---
# Exporting Scraped Player Data

In [4]:
player_db = pd.DataFrame(players).T
player_db.to_csv('../data/player_raw.csv',index=False)
player_db.drop(failed_to_qualify,inplace=True)
player_db.to_csv('../data/player_db.csv',index=False)